<a href="https://colab.research.google.com/github/khaitrinh2010/Blog-Project/blob/main/Cats_and_dogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip

--2023-03-18 10:09:37--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.143.128, 108.177.119.128, 108.177.127.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.143.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M  30.3MB/s    in 2.2s    

2023-03-18 10:09:39 (30.3 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [ ]:
import os
import zipfile

local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r') #Doc file zip
zip_ref.extractall('/tmp') #extarct vao file nao
zip_ref.close()

In [ ]:
base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Directory with our training cat pictures
train_cats_dir = os.path.join(train_dir, 'cats')

# Directory with our training dog pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')

# Directory with our validation cat pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')

# Directory with our validation dog pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')


In [ ]:
%cd /tmp


/tmp


In [ ]:
%cd /tmp/cats_and_dogs_filtered

/tmp/cats_and_dogs_filtered


In [ ]:
%cd train

/tmp/cats_and_dogs_filtered/train


In [ ]:
%cd cats

/tmp/cats_and_dogs_filtered/train/cats


In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import Model

In [ ]:
import tensorflow as tf


In [ ]:
#preprocessing data

train_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
val_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [ ]:
#flow_from_directory: ảnh từ các classes khác nhau trong các folders khác nhau nhưng từ chung một parent folder
train_generator = train_data_gen.flow_from_directory(train_dir,
                                                     target_size=(150,150),
                                                     batch_size=32,
                                                     class_mode = "binary" #predict 0 or 1
                                                     )
val_generator = val_data_gen.flow_from_directory(validation_dir,
                                                     target_size=(150,150),
                                                     batch_size=32,
                                                     class_mode = "binary"
                                                     )

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:
from tensorflow.keras import Sequential
from keras.layers import Conv2D, MaxPooling2D

from keras.layers import Activation, Dropout, Flatten, Dense

from keras import backend as K


img_width, img_height = 150, 150

In [ ]:
#Code mo hinh
model = Sequential()
input_shape = (img_width, img_height, 3)

model.add(Conv2D(32, (2, 2), input_shape=input_shape))

model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(32, (2, 2)))

model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(64, (2, 2)))

model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(64))

model.add(Activation('relu'))

model.add(Dropout(0.5))

model.add(Dense(1))

model.add(Activation('sigmoid'))

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 149, 149, 32)      416       
                                                                 
 activation (Activation)     (None, 149, 149, 32)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 73, 73, 32)        4128      
                                                                 
 activation_1 (Activation)   (None, 73, 73, 32)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 32)       0         
 2D)                                                    

In [ ]:
import tensorflow.keras.optimizers as Opt

model.compile(loss='binary_crossentropy',
              optimizer=Opt.Adam(),
              metrics=["accuracy"]
              )

In [ ]:
import numpy as np

In [ ]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=2,
      validation_data=val_generator,
      validation_steps=50,
      verbose=2)
model.save("model.h5") #steps

Epoch 1/2


<ipython-input-19-e10484146ad7>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


100/100 - 60s - loss: 0.6646 - accuracy: 0.6010 - val_loss: 0.6408 - val_accuracy: 0.5740 - 60s/epoch - 598ms/step


In [ ]:
from google.colab import files
with open("image.txt", "w") as a:
  a.write("https://thumbs.dreamstime.com/b/golden-retriever-dog-21668976.jpg")
files.download('image.txt')



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from keras.models import load_model
from tensorflow.keras.utils import load_img
with open("image.txt", 'r') as f:
  img_path = f.read()
model = load_model("model.h5")
image  = load_img(path=img_path, target_size = (150,150))
img = np.array(image)
img = img / 255.0

img = img.reshape(1,150,150,3)

label = model.predict(img)

print("Predicted Class (0 - Cars , 1- Planes): ", label[0][0])



FileNotFoundError: ignored